In [408]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [409]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastai

from sklearn import metrics
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor , RandomForestClassifier
import re
import seaborn as sns

from sklearn import metrics
import io
import xgboost as xgb

In [410]:
PATH = './data/'
!ls {PATH}

sample_submission.csv  test.csv  train.csv


In [411]:
train_raw = pd.read_csv(f'{PATH}train.csv',low_memory=False,parse_dates=['opened_at'])
test_raw = pd.read_csv(f'{PATH}test.csv',low_memory=False,parse_dates=['opened_at'])

In [412]:
train_raw.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062,INC0000063,INC0000064,INC0000065,INC0000066,INC0000067,...,INC0028188,INC0028190,INC0028191,INC0028192,INC0028193,INC0028195,INC0028196,INC0028200,INC0028201,INC0028203
reassignment_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
reopen_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
update_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,Opened by 8,Opened by 397,Opened by 8,Opened by 180,Opened by 180,Opened by 180,Opened by 131,Opened by 131,Opened by 24,Opened by 180,...,Opened by 24,Opened by 301,Opened by 131,Opened by 62,Opened by 24,Opened by 17,Opened by 17,Opened by 24,Opened by 24,Opened by 55
opened_at,2016-02-29 01:16:00,2016-02-29 04:40:00,2016-02-29 06:10:00,2016-02-29 06:38:00,2016-02-29 06:58:00,2016-02-29 07:08:00,2016-02-29 07:10:00,2016-02-29 07:38:00,2016-02-29 08:03:00,2016-02-29 08:03:00,...,2016-05-11 08:41:00,2016-05-11 08:41:00,2016-05-11 08:42:00,2016-05-11 08:44:00,2016-05-11 08:44:00,2016-05-11 08:45:00,2016-05-11 08:46:00,2016-05-11 08:50:00,2016-05-11 08:51:00,2016-05-11 08:52:00
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone
location,Location 143,Location 165,Location 204,Location 204,Location 93,Location 93,Location 143,Location 108,Location 161,Location 143,...,Location 161,Location 56,Location 161,Location 143,Location 108,Location 204,Location 93,Location 97,Location 51,Location 204
category,Category 55,Category 40,Category 20,Category 9,Category 53,Category 20,Category 53,Category 44,Category 55,Category 9,...,Category 9,Category 37,Category 42,Category 53,Category 20,Category 42,Category 37,Category 42,Category 53,Category 26


In [413]:
def extracting_num(df_raw):
  df = df_raw.copy()
  df['opened_by'] = df['opened_by'].str.split(expand=True)[2]
  df['location'] = df['location'].str.split(expand=True)[1]
  df['category'] = df['category'].str.split(expand=True)[1]
  df['subcategory'] = df['subcategory'].str.split(expand=True)[1]
  df['impact'] = df['impact'].str.split(expand=True)[0]
  df['urgency'] = df['urgency'].str.split(expand=True)[0]
  df['priority'] = df['priority'].str.split(expand=True)[0]
  df['assigned_to'] = df['assigned_to'].str.split(expand=True)[1]
  return df

In [414]:
train_raw = extracting_num(train_raw)

In [415]:
train_raw.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062,INC0000063,INC0000064,INC0000065,INC0000066,INC0000067,...,INC0028188,INC0028190,INC0028191,INC0028192,INC0028193,INC0028195,INC0028196,INC0028200,INC0028201,INC0028203
reassignment_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
reopen_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
update_count,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,8,397,8,180,180,180,131,131,24,180,...,24,301,131,62,24,17,17,24,24,55
opened_at,2016-02-29 01:16:00,2016-02-29 04:40:00,2016-02-29 06:10:00,2016-02-29 06:38:00,2016-02-29 06:58:00,2016-02-29 07:08:00,2016-02-29 07:10:00,2016-02-29 07:38:00,2016-02-29 08:03:00,2016-02-29 08:03:00,...,2016-05-11 08:41:00,2016-05-11 08:41:00,2016-05-11 08:42:00,2016-05-11 08:44:00,2016-05-11 08:44:00,2016-05-11 08:45:00,2016-05-11 08:46:00,2016-05-11 08:50:00,2016-05-11 08:51:00,2016-05-11 08:52:00
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone
location,143,165,204,204,93,93,143,108,161,143,...,161,56,161,143,108,204,93,97,51,204
category,55,40,20,9,53,20,53,44,55,9,...,9,37,42,53,20,42,37,42,53,26


In [416]:
train_raw.drop(columns='vendor',inplace=True)
train_raw.drop(columns='reassignment_count',inplace=True)
train_raw.drop(columns='reopen_count',inplace=True)
train_raw.drop(columns='update_count',inplace=True)

In [417]:
train_raw.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062,INC0000063,INC0000064,INC0000065,INC0000066,INC0000067,...,INC0028188,INC0028190,INC0028191,INC0028192,INC0028193,INC0028195,INC0028196,INC0028200,INC0028201,INC0028203
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,8,397,8,180,180,180,131,131,24,180,...,24,301,131,62,24,17,17,24,24,55
opened_at,2016-02-29 01:16:00,2016-02-29 04:40:00,2016-02-29 06:10:00,2016-02-29 06:38:00,2016-02-29 06:58:00,2016-02-29 07:08:00,2016-02-29 07:10:00,2016-02-29 07:38:00,2016-02-29 08:03:00,2016-02-29 08:03:00,...,2016-05-11 08:41:00,2016-05-11 08:41:00,2016-05-11 08:42:00,2016-05-11 08:44:00,2016-05-11 08:44:00,2016-05-11 08:45:00,2016-05-11 08:46:00,2016-05-11 08:50:00,2016-05-11 08:51:00,2016-05-11 08:52:00
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone
location,143,165,204,204,93,93,143,108,161,143,...,161,56,161,143,108,204,93,97,51,204
category,55,40,20,9,53,20,53,44,55,9,...,9,37,42,53,20,42,37,42,53,26
subcategory,170,215,125,97,168,125,168,229,185,110,...,164,123,223,303,174,223,174,223,130,275
impact,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
urgency,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [418]:
train_raw['subcategory'].value_counts()

174    5528
223    2837
175    1524
164     962
9       698
       ... 
213       1
140       1
282       1
212       1
35        1
Name: subcategory, Length: 214, dtype: int64

In [419]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train_raw['notify']= labelencoder.fit_transform(train_raw['notify'])

In [420]:
df = pd.DataFrame(train_raw['assigned_to'])

In [421]:
train_raw['assigned_to']=df.fillna(13)
train_raw['opened_by']=df.fillna(17)
train_raw['location']=df.fillna(204)
train_raw['category']=df.fillna(42)
train_raw['subcategory']=df.fillna(174)

In [422]:
train_raw.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062,INC0000063,INC0000064,INC0000065,INC0000066,INC0000067,...,INC0028188,INC0028190,INC0028191,INC0028192,INC0028193,INC0028195,INC0028196,INC0028200,INC0028201,INC0028203
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,17,89,17,125,17,17,78,216,17,236,...,24,82,194,33,93,13,83,17,17,17
opened_at,2016-02-29 01:16:00,2016-02-29 04:40:00,2016-02-29 06:10:00,2016-02-29 06:38:00,2016-02-29 06:58:00,2016-02-29 07:08:00,2016-02-29 07:10:00,2016-02-29 07:38:00,2016-02-29 08:03:00,2016-02-29 08:03:00,...,2016-05-11 08:41:00,2016-05-11 08:41:00,2016-05-11 08:42:00,2016-05-11 08:44:00,2016-05-11 08:44:00,2016-05-11 08:45:00,2016-05-11 08:46:00,2016-05-11 08:50:00,2016-05-11 08:51:00,2016-05-11 08:52:00
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone
location,204,89,204,125,204,204,78,216,204,236,...,24,82,194,33,93,13,83,17,204,204
category,42,89,42,125,42,42,78,216,42,236,...,24,82,194,33,93,13,83,17,42,42
subcategory,174,89,174,125,174,174,78,216,174,236,...,24,82,194,33,93,13,83,17,174,174
impact,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
urgency,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [423]:
for index,row in train_raw.iterrows() :
    if(row['target_days']>10):
        train_raw.drop(index,inplace=True)
for index,row in train_raw.iterrows() :
    if(row['target_days']<=4):
        train_raw.drop(index,inplace=True)
# for index,row in train_raw.iterrows() :
#     if(row['knowledge']==True):
#         train_raw.drop(index,inplace=True)

In [424]:
train_raw['made_sla'].value_counts()

True    13130
Name: made_sla, dtype: int64

In [425]:
df_train = train_raw.copy()
add_datepart(df_train,'opened_at')
df_train.T

,0,1,2,3,4,5,6,7,8,9,...,19987,19988,19990,19992,19993,19994,19995,19996,19997,19999
Id,INC0000045,INC0000047,INC0000057,INC0000060,INC0000062,INC0000063,INC0000064,INC0000065,INC0000066,INC0000067,...,INC0028185,INC0028186,INC0028188,INC0028191,INC0028192,INC0028193,INC0028195,INC0028196,INC0028200,INC0028203
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,17,89,17,125,17,17,78,216,17,236,...,17,17,24,194,33,93,13,83,17,17
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone
location,204,89,204,125,204,204,78,216,204,236,...,17,17,24,194,33,93,13,83,17,204
category,42,89,42,125,42,42,78,216,42,236,...,17,17,24,194,33,93,13,83,17,42
subcategory,174,89,174,125,174,174,78,216,174,236,...,17,17,24,194,33,93,13,83,17,174
impact,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
urgency,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
priority,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [426]:
df_train['target_days'].value_counts()

5     8881
6     1276
7      878
8      731
10     712
9      652
Name: target_days, dtype: int64

In [427]:
df_train.drop(columns='Id',inplace=True)

In [428]:
df_train.drop(columns='opened_atYear',inplace=True)
# df_train.drop(columns='made_sla',inplace=True)
df_train.drop(columns='impact',inplace=True)
df_train.drop(columns='urgency',inplace=True)
df_train.drop(columns='opened_atIs_year_start',inplace=True)
df_train.drop(columns='opened_atIs_year_end',inplace=True)

In [429]:
df_train.T

,0,1,2,3,4,5,6,7,8,9,...,19987,19988,19990,19992,19993,19994,19995,19996,19997,19999
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,17,89,17,125,17,17,78,216,17,236,...,17,17,24,194,33,93,13,83,17,17
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone
location,204,89,204,125,204,204,78,216,204,236,...,17,17,24,194,33,93,13,83,17,204
category,42,89,42,125,42,42,78,216,42,236,...,17,17,24,194,33,93,13,83,17,42
subcategory,174,89,174,125,174,174,78,216,174,236,...,17,17,24,194,33,93,13,83,17,174
priority,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
assigned_to,13,89,13,125,13,13,78,216,13,236,...,17,17,24,194,33,93,13,83,17,13
knowledge,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
notify,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [430]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [431]:
# df_train['knowledge']= label_encoder.fit_transform(df_train['knowledge'])
# df_train['opened_atIs_month_end']= label_encoder.fit_transform(df_train['opened_atIs_month_end'])
# df_train['opened_atIs_month_start']= label_encoder.fit_transform(df_train['opened_atIs_month_start'])
# df_train['opened_atIs_quarter_end']= label_encoder.fit_transform(df_train['opened_atIs_quarter_end'])
# df_train['opened_atIs_quarter_start']= label_encoder.fit_transform(df_train['opened_atIs_quarter_start'])
df_train['contact_type']= label_encoder.fit_transform(df_train['contact_type'])

In [432]:
df_train.isnull().sum()

made_sla                     0
opened_by                    0
contact_type                 0
location                     0
category                     0
subcategory                  0
priority                     0
assigned_to                  0
knowledge                    0
notify                       0
target_days                  0
opened_atMonth               0
opened_atWeek                0
opened_atDay                 0
opened_atDayofweek           0
opened_atDayofyear           0
opened_atIs_month_end        0
opened_atIs_month_start      0
opened_atIs_quarter_end      0
opened_atIs_quarter_start    0
opened_atElapsed             0
dtype: int64

In [433]:
X = df_train.copy()

In [434]:
X.drop(columns='target_days',inplace=True)

In [435]:
y = df_train['target_days']

In [436]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.00005, random_state = 0)

In [437]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_valid = sc.transform(X_valid)

In [438]:
def print_score(m):
    print(metrics.mean_absolute_error(m.predict(X_train),y_train),metrics.mean_absolute_error(m.predict(X_valid),y_valid))

In [439]:
# from sklearn.neighbors import KNeighborsClassifier
# n = KNeighborsClassifier()
# n.fit(X_train,y_train)
# print_score(n)

In [440]:
# from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]# Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
# print(random_grid)

In [441]:
# rf = RandomForestRegressor()

In [442]:
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [443]:
# rf_random.fit(X_train,y_train)
# print_score(rf_random)

In [444]:
# rf_random.best_params_

In [445]:
from sklearn.ensemble import RandomForestRegressor , RandomForestClassifier
# m = RandomForestClassifier(n_jobs=-1,random_state=3,min_samples_leaf=1)
# m.fit(X_train,y_train)
# print_score(m)

In [446]:
# k = RandomForestRegressor(n_jobs=-1,random_state=3,n_estimators=1870,min_samples_split=2,min_samples_leaf=4,max_features='sqrt',max_depth=80,bootstrap=True)
# k.fit(X_train,y_train)
# print_score(k)

0.6139322356767599 0.43156130799754067


In [447]:
# df_train.T

In [448]:
# import keras
# from keras.models import Sequential
# from keras.layers import Dense

In [449]:
# classifier = Sequential()
# classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu', input_dim = 19))
# classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear'))
# classifier.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['accuracy'])

In [450]:
# classifier.fit(X_train, y_train, batch_size = 20, epochs = 50)

In [451]:
# print_score(classifier)

In [452]:
Id = test_raw['Id'].copy()

In [453]:
test_raw.shape

(4918, 18)

In [454]:
test_raw = extracting_num(test_raw)

In [455]:
test_raw.drop(columns='vendor',inplace=True)
test_raw.drop(columns='reassignment_count',inplace=True)
test_raw.drop(columns='reopen_count',inplace=True)
test_raw.drop(columns='update_count',inplace=True)

In [456]:
df = pd.DataFrame(test_raw['assigned_to'])

In [457]:
test_raw['assigned_to']=df.fillna(13)
test_raw['opened_by']=df.fillna(17)
test_raw['location']=df.fillna(204)
test_raw['category']=df.fillna(42)
test_raw['subcategory']=df.fillna(174)

In [458]:
test_raw.T

,0,1,2,3,4,5,6,7,8,9,...,4908,4909,4910,4911,4912,4913,4914,4915,4916,4917
Id,INC0028205,INC0028208,INC0028209,INC0028210,INC0028211,INC0028212,INC0028213,INC0028214,INC0028215,INC0028216,...,INC0119457,INC0119983,INC0119987,INC0120268,INC0120303,INC0120304,INC0120319,INC0120495,INC0120835,INC0121064
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,249,233,219,7,76,17,125,140,33,17,...,17,17,17,17,17,6,17,17,10,10
opened_at,2016-05-11 08:52:00,2016-05-11 08:53:00,2016-05-11 08:55:00,2016-05-11 08:59:00,2016-05-11 08:59:00,2016-05-11 08:59:00,2016-05-11 09:00:00,2016-05-11 09:00:00,2016-05-11 09:01:00,2016-05-11 09:01:00,...,2017-02-13 11:31:00,2017-02-14 11:11:00,2017-02-14 11:15:00,2017-02-14 18:31:00,2017-02-15 01:52:00,2017-02-15 02:02:00,2017-02-15 07:09:00,2017-02-15 11:58:00,2017-02-16 09:09:00,2017-02-16 14:17:00
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,IVR,IVR,IVR,IVR,IVR,Email,IVR,Phone,Email,Email
location,249,233,219,7,76,204,125,140,33,204,...,204,204,204,204,204,6,204,204,10,10
category,249,233,219,7,76,42,125,140,33,42,...,42,42,42,42,42,6,42,42,10,10
subcategory,249,233,219,7,76,174,125,140,33,174,...,174,174,174,174,174,6,174,174,10,10
impact,2,2,2,2,2,2,2,2,2,2,...,3,3,3,3,3,2,3,2,2,2
urgency,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [459]:
df_test = test_raw.copy()
add_datepart(df_test,'opened_at')
df_test.T

,0,1,2,3,4,5,6,7,8,9,...,4908,4909,4910,4911,4912,4913,4914,4915,4916,4917
Id,INC0028205,INC0028208,INC0028209,INC0028210,INC0028211,INC0028212,INC0028213,INC0028214,INC0028215,INC0028216,...,INC0119457,INC0119983,INC0119987,INC0120268,INC0120303,INC0120304,INC0120319,INC0120495,INC0120835,INC0121064
made_sla,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
opened_by,249,233,219,7,76,17,125,140,33,17,...,17,17,17,17,17,6,17,17,10,10
contact_type,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,Phone,...,IVR,IVR,IVR,IVR,IVR,Email,IVR,Phone,Email,Email
location,249,233,219,7,76,204,125,140,33,204,...,204,204,204,204,204,6,204,204,10,10
category,249,233,219,7,76,42,125,140,33,42,...,42,42,42,42,42,6,42,42,10,10
subcategory,249,233,219,7,76,174,125,140,33,174,...,174,174,174,174,174,6,174,174,10,10
impact,2,2,2,2,2,2,2,2,2,2,...,3,3,3,3,3,2,3,2,2,2
urgency,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
priority,3,3,3,3,3,3,3,3,3,3,...,4,4,4,4,4,3,4,3,3,3


In [460]:
# df_test['knowledge']= label_encoder.fit_transform(df_test['knowledge'])
# df_test['opened_atIs_month_end']= label_encoder.fit_transform(df_test['opened_atIs_month_end'])
# df_test['opened_atIs_month_start']= label_encoder.fit_transform(df_test['opened_atIs_month_start'])
# df_test['opened_atIs_quarter_end']= label_encoder.fit_transform(df_test['opened_atIs_quarter_end'])
# df_test['opened_atIs_quarter_start']= label_encoder.fit_transform(df_test['opened_atIs_quarter_start'])
df_test['contact_type']= label_encoder.fit_transform(df_test['contact_type'])

In [461]:
df_test.drop(columns='Id',inplace=True)
df_test.drop(columns='opened_atYear',inplace=True)
# df_test.drop(columns='made_sla',inplace=True)
df_test.drop(columns='impact',inplace=True)
df_test.drop(columns='urgency',inplace=True)
df_test.drop(columns='opened_atIs_year_start',inplace=True)
df_test.drop(columns='opened_atIs_year_end',inplace=True)

In [462]:
df_test['notify']= labelencoder.fit_transform(df_test['notify'])

In [463]:
y_test = k.predict(df_test).astype(int)

In [464]:
y_test

array([5, 5, 5, ..., 6, 6, 6])

In [465]:
dict = {
    'Id':Id,
    'target_days':y_test
}

In [466]:
submit = pd.DataFrame(dict)
submit['target_days'].value_counts()

5    3859
6    1012
7      47
Name: target_days, dtype: int64

In [467]:
submit.to_csv('submit12.csv',header=True,index=False)

In [468]:
# submit2 = pd.DataFrame(y_test,Id,columns=["target_days"])

In [469]:
# submit2['target_days'].value_counts()